In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 12 × Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 1 on 12 virtual cores


## 6-1. イテレーションの仕組み

### 6-1-1. イテレーションも糖衣構文

#### 仮想コード6-1. `for` 文の例

```julia
for value in iter
    println(value)
end
```

#### 仮想コード6-2. `for` 文と等価なコード例

```julia
next = iterate(iter)
while next !== nothing
    value, state = next
    println(value)
    next = iterate(iter, state)
end
```

#### コード6-1. 仮想コード6-1./6-2. の動作確認

In [2]:
iter = [31, 41, 59, 26, 53];

In [3]:
for value in iter
    println(value)
end

31
41
59
26
53


In [4]:
next = iterate(iter);

while next !== nothing
    value, state = next
    println(value)
    next = iterate(iter, state)
end

31
41
59
26
53


### 6-1-2. `iterate()` 関数

#### 仮想コード6-2. `for` 文と等価なコード例（再掲）

```julia
next = iterate(iter)
while next !== nothing
    value, state = next
    println(value)
    next = iterate(iter, state)
end
```

#### 仮想コード6-3. `AbstractArray` の `iterate()` 関数の実装イメージ

```julia
function Base.iterate(a::AbstractArray)
    isempty(a) && return nothing
    (a[1], 2)
end
function Base.iterate(a::AbstractArray, i)
    i > length(a) && return nothing
    (a[i], i + 1)
end
```

#### 仮想コード6-4. `AbstractString` の `iterate()` 関数の実装イメージ

```julia
function Base.iterate(s::AbstractString)
    isempty(a) && return nothing
    (s[1], nextind(s, 1))
end
function Base.iterate(s::AbstractString, i)
    i > ncodeunits(s) && return nothing
    (s[i], nextind(s, i))
end
```

#### コード6-2. `iterate()` 関数の動作確認（配列、文字列）

In [5]:
iterate([31, 41, 59, 26, 53])

(31, 2)

In [6]:
iterate([31, 41, 59, 26, 53], 2)

(41, 3)

In [7]:
iterate([31, 41, 59, 26, 53], 5)

(53, 6)

In [8]:
iterate([31, 41, 59, 26, 53], 6) === nothing

true

In [9]:
iterate("あいうえお")

('あ', 4)

In [10]:
iterate("あいうえお", 4)

('い', 7)

In [11]:
iterate("あいうえお", 13)

('お', 16)

In [12]:
iterate("あいうえお", 16) === nothing

true

#### コード6-3(1). `GeometricSequence.jl`（1～18行目）

In [13]:
struct GeometricSequence{T<:Number} <: AbstractVector{T}
    a::T
    r::T
    n::Int
end

function GeometricSequence(a::T1, r::T2, n::Integer) where {T1<:Number, T2<:Number}
    GeometricSequence(promote(a, r)..., Int(n))
end

Base.length(seq::GeometricSequence) = seq.n
Base.size(seq::GeometricSequence) = (seq.n,)
Base.getindex(seq::GeometricSequence, index::Integer) = seq.a * seq.r ^ (index - 1)

function Base.show(io::IO, seq::GeometricSequence)
    print(io, "GeometricSequence(", seq.a, ", ", seq.r, ", ", seq.n, ")")
end
Base.show(io::IO, ::MIME"text/plain", seq::GeometricSequence) = show(io, seq)

#### コード6-3(2). `GeometricSequence.jl`（37～44行目）

In [14]:
function Base.iterate(seq::GeometricSequence)
    seq.n > 0 || return nothing
    (seq.a, (seq.a * seq.r, seq.n - 1))
end
function Base.iterate(seq::GeometricSequence, (next, count)::NTuple{2})
    count == 0 && return nothing
    (next, (next * seq.r, count - 1))
end

In [15]:
include("GeometricSequence.jl")

#### コード6-4. `GeometricSequence` のイテレーション動作確認

In [16]:
seq0 = GeometricSequence(1, 3, 5);

In [17]:
for value in seq0
    println(value)
end

1
3
9
27
81


In [18]:
for i = 1:length(seq0)
    println(seq0[i])
end  # 参考：出力結果が同じになることの確認のため

1
3
9
27
81


In [19]:
seq1 = GeometricSequence(1, 0.5, 20);

In [20]:
@time for i = 1:length(seq1)
    println(devnull, seq1[i])
end

  0.025670 seconds (25.87 k allocations: 1.408 MiB, 99.58% compilation time)


In [21]:
@time for value in seq1
    println(devnull, value)
end

  0.004581 seconds (5.06 k allocations: 288.839 KiB, 97.93% compilation time)


#### コード6-5. `GeometricSequence` の分割代入動作確認

In [22]:
seq0 = GeometricSequence(1, 3, 5);

In [23]:
x₁, x₂, x₃, x₄, x₅ = seq0

GeometricSequence(1, 3, 5)

In [24]:
(x₁, x₂, x₃, x₄, x₅)

(1, 3, 9, 27, 81)

In [25]:
x₁, x₂, x₃ = seq0

GeometricSequence(1, 3, 5)

In [26]:
(x₁, x₂, x₃)

(1, 3, 9)

In [27]:
x₁, x₂, x₃, x₄, x₅, x₆ = seq0  # エラー

LoadError: BoundsError: attempt to access 5-element GeometricSequence{Int64} at index [6]

In [28]:
struct NotIterable end
x₁, x₂, x₃ = NotIterable()  # エラー

LoadError: MethodError: no method matching iterate(::NotIterable)
[0mClosest candidates are:
[0m  iterate([91m::Union{LinRange, StepRangeLen}[39m) at range.jl:872
[0m  iterate([91m::Union{LinRange, StepRangeLen}[39m, [91m::Integer[39m) at range.jl:872
[0m  iterate([91m::T[39m) where T<:Union{Base.KeySet{<:Any, <:Dict}, Base.ValueIterator{<:Dict}} at dict.jl:712
[0m  ...

#### コード6-6. `GeometricSequence` の `in`演算子（`in()`関数） 動作確認

In [29]:
seq2 = GeometricSequence(2, 3, 20);

In [30]:
54 in seq2

true

In [31]:
486 ∈ seq2

true

In [32]:
888 ∉ seq2

true

In [33]:
seq2 ∋ 9565938

true

In [34]:
seq2 ∌ 2147483647

true

In [35]:
filter(in(seq2), 100:200)

1-element Vector{Int64}:
 162

### 6-1-3. `Base.IteratorSize` と `Base.IteratorEltype`

#### コード6-7. `GeometricSequence` の内包表記および `collect()`関数 動作確認

In [36]:
[i for i in GeometricSequence(1, 3, 5)]

5-element Vector{Int64}:
  1
  3
  9
 27
 81

In [37]:
collect(GeometricSequence(1, 3, 5))

5-element Vector{Int64}:
  1
  3
  9
 27
 81

#### コード6-8. 約数を列挙するイテレータの定義例（`Divisors1`）

In [38]:
struct Divisors1{I <: Integer}
    n::I
end

In [39]:
Base.iterate(ds::Divisors1{I}) where {I} = (one(I), one(I))

function Base.iterate(ds::Divisors1{I}, next::I) where {I}
    while next < ds.n
        next += one(I)
        ds.n % next == 0 && return (next, next)
    end
    nothing
end

#### コード6-9. `Divisors1` の動作確認（`for`文）

In [40]:
ds1_36 = Divisors1(36)

Divisors1{Int64}(36)

In [41]:
for n in ds1_36
    println(n)
end

1
2
3
4
6
9
12
18
36


#### コード6-10. `Divisors1` の動作確認（内包表記/`collect`関数、エラー例）

In [42]:
[n for n in ds1_36]

LoadError: MethodError: no method matching length(::Divisors1{Int64})
[0mClosest candidates are:
[0m  length([91m::Union{Base.KeySet, Base.ValueIterator}[39m) at abstractdict.jl:58
[0m  length([91m::Union{LinearAlgebra.Adjoint{T, S}, LinearAlgebra.Transpose{T, S}} where {T, S}[39m) at ~/julia-1.8.5/share/julia/stdlib/v1.8/LinearAlgebra/src/adjtrans.jl:172
[0m  length([91m::Union{ZMQ._Message, Base.RefValue{ZMQ._Message}}[39m) at ~/.julia/packages/ZMQ/lrABE/src/_message.jl:31
[0m  ...

In [43]:
collect(ds1_36)

LoadError: MethodError: no method matching length(::Divisors1{Int64})
[0mClosest candidates are:
[0m  length([91m::Union{Base.KeySet, Base.ValueIterator}[39m) at abstractdict.jl:58
[0m  length([91m::Union{LinearAlgebra.Adjoint{T, S}, LinearAlgebra.Transpose{T, S}} where {T, S}[39m) at ~/julia-1.8.5/share/julia/stdlib/v1.8/LinearAlgebra/src/adjtrans.jl:172
[0m  length([91m::Union{ZMQ._Message, Base.RefValue{ZMQ._Message}}[39m) at ~/.julia/packages/ZMQ/lrABE/src/_message.jl:31
[0m  ...

#### `Base.IteratorSize`

##### コード6-11. `Divisors1` が `Base.SizeUnknown` トレイトを準拠するようにした場合の動作例

In [44]:
Base.IteratorSize(::Type{<:Divisors1}) = Base.SizeUnknown()

In [45]:
ds1_36 = Divisors1(36);

In [46]:
[n for n in ds1_36]

9-element Vector{Int64}:
  1
  2
  3
  4
  6
  9
 12
 18
 36

In [47]:
collect(ds1_36)

9-element Vector{Any}:
  1
  2
  3
  4
  6
  9
 12
 18
 36

##### コード6-12. 約数を列挙するイテレータの定義例（`Divisors2`）

In [48]:
struct Divisors2{I <: Integer}
    n::I
end

In [49]:
Base.iterate(ds::Divisors2{I}) where {I} = (one(I), one(I))
function Base.iterate(ds::Divisors2{I}, next::I) where {I}
    while next < ds.n
        next += one(I)
        ds.n % next == 0 && return (next, next)
    end
    nothing
end

In [50]:
# ↓デフォルトでこうなっているので明示的に多重定義せず
# Base.IteratorSize(::Type{<:Divisors2}) = Base.HasLength()

function Base.length(ds::Divisors2)
    n = ds.n
    sqrtn = isqrt(n)
    l = trailing_zeros(n) + 1
    n >>= l - 1
    p = oftype(n, 3)
    d = 2
    while p ≤ n && p ≤ sqrtn
        if n % p == 0
            cnt_1 = 1
            while n % p == 0
                n ÷= p
                cnt_1 += 1
            end
            l *= cnt_1
        end
        p += d
        d = 6 - d
    end
    n > 1 ? 2l : l
end

##### コード6-13. `Divisors2` の動作例

In [51]:
ds2_36 = Divisors2(36)

Divisors2{Int64}(36)

In [52]:
Base.IteratorSize(typeof(ds2_36))

Base.HasLength()

In [53]:
length(ds2_36)

9

In [54]:
for n in ds2_36
    println(n)
end

1
2
3
4
6
9
12
18
36


In [55]:
[n for n in ds2_36]

9-element Vector{Int64}:
  1
  2
  3
  4
  6
  9
 12
 18
 36

In [56]:
collect(ds2_36)

9-element Vector{Any}:
  1
  2
  3
  4
  6
  9
 12
 18
 36

##### コード6-14. `Divisors1` と `Divisors2` のパフォーマンス比較その1（`Base.IteratorSize` の差異）

In [57]:
ds1_360 = Divisors1(360);
ds2_360 = Divisors2(360);

In [58]:
Base.IteratorSize(typeof(ds1_360))

Base.SizeUnknown()

In [59]:
Base.IteratorSize(typeof(ds2_360))

Base.HasLength()

In [60]:
[n for n in ds1_360]

24-element Vector{Int64}:
   1
   2
   3
   4
   5
   6
   8
   9
  10
  12
  15
  18
  20
  24
  30
  36
  40
  45
  60
  72
  90
 120
 180
 360

In [61]:
[n for n in ds2_360]

24-element Vector{Int64}:
   1
   2
   3
   4
   5
   6
   8
   9
  10
  12
  15
  18
  20
  24
  30
  36
  40
  45
  60
  72
  90
 120
 180
 360

In [62]:
[n for n in ds1_360] == [n for n in ds2_360]

true

In [63]:
@time [n for n in ds1_360];

  0.000010 seconds (4 allocations: 496 bytes)


In [64]:
@time [n for n in ds2_360];

  0.000009 seconds (2 allocations: 272 bytes)


#### `Base.IteratorEltype`

##### コード6-15. `Base.eltype()` を適切に定義した `Divisors1` および `Divisors2` の挙動確認

In [65]:
Base.eltype(::Type{Divisors1{I}}) where {I} = I
Base.eltype(::Type{Divisors2{I}}) where {I} = I

In [66]:
ds1_36 = Divisors1(36)

Divisors1{Int64}(36)

In [67]:
ds2_36 = Divisors2(36)

Divisors2{Int64}(36)

In [68]:
Base.IteratorEltype(typeof(ds1_36))

Base.HasEltype()

In [69]:
Base.eltype(ds1_36)

Int64

In [70]:
Base.IteratorEltype(typeof(ds2_36))

Base.HasEltype()

In [71]:
Base.eltype(ds2_36)

Int64

In [72]:
collect(ds1_36)

9-element Vector{Int64}:
  1
  2
  3
  4
  6
  9
 12
 18
 36

In [73]:
collect(ds2_36)

9-element Vector{Int64}:
  1
  2
  3
  4
  6
  9
 12
 18
 36

#### コード6-16. `Divisors3` の定義（`Base.SizeUnknown`/`Base.EltypeUnknown`）

In [74]:
struct Divisors3{I <: Integer}
    n::I
end

In [75]:
Base.iterate(ds::Divisors3{I}) where {I} = (one(I), one(I))
function Base.iterate(ds::Divisors3{I}, next::I) where {I}
    while next < ds.n
        next += one(I)
        ds.n % next == 0 && return (next, next)
    end
    nothing
end

In [76]:
Base.IteratorSize(::Type{<:Divisors3}) = Base.SizeUnknown()

In [77]:
Base.IteratorEltype(::Type{<:Divisors3}) = Base.EltypeUnknown()

##### コード6-17. `Divisors3` の挙動確認

In [78]:
ds3_36 = Divisors3(36)

Divisors3{Int64}(36)

In [79]:
[n for n in ds3_36]

9-element Vector{Int64}:
  1
  2
  3
  4
  6
  9
 12
 18
 36

In [80]:
collect(ds3_36)

9-element Vector{Int64}:
  1
  2
  3
  4
  6
  9
 12
 18
 36

##### コード6-18. `Divisors1` と `Divisors3` のパフォーマンス比較（`Base.IteratorEltype` の差異による比較）

In [81]:
ds1_360 = Divisors1(360);
ds3_360 = Divisors3(360);

In [82]:
Base.IteratorEltype(typeof(ds1_360))

Base.HasEltype()

In [83]:
Base.eltype(ds1_36)

Int64

In [84]:
Base.IteratorEltype(typeof(ds3_360))

Base.EltypeUnknown()

In [85]:
collect(ds1_360)

24-element Vector{Int64}:
   1
   2
   3
   4
   5
   6
   8
   9
  10
  12
  15
  18
  20
  24
  30
  36
  40
  45
  60
  72
  90
 120
 180
 360

In [86]:
collect(ds3_360)

24-element Vector{Int64}:
   1
   2
   3
   4
   5
   6
   8
   9
  10
  12
  15
  18
  20
  24
  30
  36
  40
  45
  60
  72
  90
 120
 180
 360

In [87]:
collect(ds1_360) == collect(ds3_360)

true

In [88]:
@time collect(ds1_360);

  0.000007 seconds (3 allocations: 480 bytes)


In [89]:
@time collect(ds3_360);

  0.000006 seconds (3 allocations: 480 bytes)
